# Setup

In [ ]:
# Image from Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --quiet -U --no-cache-dir gdown --pre
!pip install --quiet nltk
!pip install --quiet emoji
!pip install --quiet imblearn
!pip install --quiet gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.7/353.7 kB 21.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install --q Sastrawi
!pip install --quiet nlp-id

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 54.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 52.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


In [ ]:
# from nlp_id.stopword import StopWord
# from nlp_id.postag import PosTag
# from nlp_id.lemmatizer import Lemmatizer

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [ ]:
import emoji
import string
import re
import numpy as np
import pandas as pd

import seaborn as sns # untuk visualisasi
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline

# potongan kode di bawah adalah agar gambar graph yang 
# dihasilkan mempunyai kualitas yang cukup baik
sns.set(rc={"figure.dpi":100, 'savefig.dpi':300})
sns.set_context('notebook')
sns.set_style("ticks")

In [ ]:
abusive_lexicon = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/abusive.csv')
kamus_alay = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/kamusalay.csv', encoding='latin-1')
most_freq = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/most_freq_words.csv', encoding='latin-1')
potensi_bias = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/potensi_bias.csv', encoding='latin-1')
replacement_data = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/bias_replacement.csv', encoding='latin-1')

all_test_data = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/all_extracted_test_data.csv', encoding='latin-1')
all_synthetic_test_data = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/all_extracted_synthetic_test_data.csv', encoding='latin-1')
all_train_data = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/all_extracted_train_data.csv', encoding='latin-1')

custom_data = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/custom_data.csv', encoding='latin-1')

In [ ]:
sw_file = '/content/drive/MyDrive/Tugas Akhir/Data/nlp_id_stopwords.txt'

with open(sw_file) as file:
    stopwords_id = [line.rstrip() for line in file]

In [ ]:
# import StemmerFactory class
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemming process
sentence = 'Perekonomian Indonesia sedang dalam pertumbuhan yang membanggakan'
output   = stemmer.stem(sentence)

print(output)
# ekonomi indonesia sedang dalam tumbuh yang bangga

print(stemmer.stem('meniru-nirukannya'))
# mereka tiru

In [ ]:
# Tokenization & Emoticon Processor

regex_str = []
regex_str.append(r'<[^>]+>') # HTML tags
regex_str.append(r'(?:@[\w_]+)') # @-mentions
regex_str.append(r'(?:&[\w_]+)')
regex_str.append(r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)") # hash-tags
regex_str.append(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+') # URLs
regex_str.append(r'(?:(?:\d+,?)+(?:\.?\d+)?)') # numbers
regex_str.append(r"(?:[a-z][a-z'\-_]+[a-z])") # words with - and '
regex_str.append(r'(?:[\w_]+)') # other words
regex_str.append(r'(?:\S)') # anything else

tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)

def tokenize(sentence):
    return [emoji.demojize(term) for term in tokens_re.findall(sentence)]

In [ ]:
# Stopwords Removal & Lemmatizing

punctuation = list(string.punctuation)
stops = stopwords_id + punctuation + ['rt', 'via', '…','•','“', 'user', 'url',
                                                  'xf0', 'x9f', 'x98', 'x84', 'xf0', 'x9f', 'x98', 'x84', 'xf0', 'x9f', 
                                                  'x98', 'x84', 'xf', 'xe2', 'x80', 'x94', 'x82', 'xa4', 'x9d', 'xc2',
                                                  'x86', 'x85', 'x81', 'xa6', 'xa3', 'xad', 'x91', 'x81', 'xe3', 'x99',
                                                  'x', 'a', 'n', 'b',
                                                  '1', '2', '3', '4', '5', '6', '7', '8', '9', '0']

def remove_stops_and_lemmatize(tokenized_sentence):
  processed_words = []
  for word in tokenized_sentence:
    if word.lower() in kamus_alay.Alay.values:
      row = kamus_alay[kamus_alay['Alay'] == word.lower()]
      word = row['Tidak Alay'].values[0]
    if word.lower() not in stops:
      processed_words.append(stemmer.stem(word))
  return processed_words

In [ ]:
def remove_stops_and_lemmatize_keep_exclamation(tokenized_sentence):
  processed_words = []
  for word in tokenized_sentence:
    if word == "!":
      processed_words.append(word
    if word.lower() not in stops:
      processed_words.append(stemmer.stem(word))
  return processed_words

# Code Copy: Bias Replacement Data

In [ ]:
custom_data

In [ ]:
syn_df = custom_data.copy()[0:0]
syn_df

In [ ]:
def replace_bias_alay(sentence):
  sen = sentence
  tokenized_sentence = tokenize(sentence)
  for word in tokenized_sentence:
    if word.lower() in kamus_alay.Alay.values:
      row = kamus_alay[kamus_alay['Alay'] == word.lower()]
      rep = row['Tidak Alay'].values[0]
      if rep in replacement_data["0"].values:
        sen = sen.replace(word, rep)
  return sen

In [ ]:
import random
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

for i, row in custom_data.iterrows():
  new_row = row
  words = remove_stops_and_lemmatize(tokenize(row["Tweet"]))
  bias_list = []
  pool = ['r1', 'r2', 'r3', 'r4', 'r5']

  sentence = replace_bias_alay(row.Tweet)

  tweet_1 = sentence.lower()
  tweet_2 = sentence.lower()
  tweet_3 = sentence.lower()
  tweet_4 = sentence.lower()
  tweet_5 = sentence.lower()

  copy_row_1 = row.copy()
  copy_row_2 = row.copy()
  copy_row_3 = row.copy()
  copy_row_4 = row.copy()
  copy_row_5 = row.copy()

  for word in words:
    if word.lower() in replacement_data["0"].values:
      bias_list.append(word)
    elif word.lower() == "komisi berantas korupsi":
      bias_list.append("kpk")
    elif word.lower() == "susilo bambang yudhoyono":
      bias_list.append("sby")
    elif word.lower() == "dewan wakil rakyat":
      bias_list.append("dpr")

  for bias in bias_list:
    bias_row = replacement_data[replacement_data['0'] == bias.lower()]
    random.shuffle(pool)

    tweet_1 = tweet_1.replace(bias, str(bias_row[pool[0]].values[0]))
    tweet_2 = tweet_2.replace(bias, str(bias_row[pool[1]].values[0]))
    tweet_3 = tweet_3.replace(bias, str(bias_row[pool[2]].values[0]))
    tweet_4 = tweet_4.replace(bias, str(bias_row[pool[3]].values[0]))
    tweet_5 = tweet_5.replace(bias, str(bias_row[pool[4]].values[0]))

  copy_row_1.Tweet = tweet_1
  copy_row_2.Tweet = tweet_2
  copy_row_3.Tweet = tweet_3
  copy_row_4.Tweet = tweet_4
  copy_row_5.Tweet = tweet_5

  syn_df = syn_df.append(copy_row_1)
  syn_df = syn_df.append(copy_row_2)
  syn_df = syn_df.append(copy_row_3)
  syn_df = syn_df.append(copy_row_4)
  syn_df = syn_df.append(copy_row_5)

In [ ]:
syn_df

In [ ]:
from google.colab import files
# syn_df.to_csv('synthetic_custom_data.csv', encoding = 'latin-1') 
# files.download('synthetic_custom_data.csv')

# Code Copy: Extract All Features

In [ ]:
syn_cus_data = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/synthetic_custom_data.csv', encoding='latin-1')

In [ ]:
syn_cus_data

In [ ]:
def set_placeholders(source_df):
  tweet = source_df["Tweet"]
  placeholder = [0 for x in range(len(tweet))]

  out_df = pd.DataFrame(tweet, columns=["Tweet"])
  out_df["Exclamation"] = placeholder
  out_df["Abusive"] = placeholder
  out_df["Length"] = placeholder
  out_df["Words"] = placeholder
  out_df["Uppercase"] = placeholder
  out_df["All Caps"] = placeholder

  for word in potensi_bias['0'].values:
      out_df[word] = placeholder
  for abusive in abusive_lexicon["ABUSIVE"].values:
      out_df[abusive] = placeholder
  for word in most_freq['0'].values:
      out_df[word] = placeholder

  out_df["HS"] = source_df["HS"]
  out_df["A"] = source_df["A"]

  for col in out_df.columns:
    if col == 'Tweet':
      continue
    out_df[col] = out_df[col].astype(int)

  return out_df

In [ ]:
import warnings
warnings.simplefilter(action='ignore')

In [ ]:
custom_df = set_placeholders(custom_data)

In [ ]:
syncus_df = set_placeholders(syn_cus_data)

In [ ]:
for i, row in custom_df.iterrows():
  words = remove_stops_and_lemmatize(tokenize(row["Tweet"]))
  ex_words = remove_stops_and_lemmatize_keep_exclamation(tokenize(row["Tweet"]))

  for xword in ex_words:
    if xword == "!":
      custom_df.loc[i, 'Exclamation'] = 1
      break

  for word in words:
      if word.lower() in abusive_lexicon.values:
        custom_df.loc[i, 'Abusive'] = 1

  length = len(row["Tweet"])
  custom_df.loc[i, 'Length'] = length

  word_count = len(words)
  custom_df.loc[i, 'Words'] = word_count

  uppercase = sum(1 for c in row["Tweet"] if c.isupper())
  custom_df.loc[i, 'Uppercase'] = uppercase

  syncus_df.loc[i*5, 'Uppercase'] = uppercase
  syncus_df.loc[i*5 +1, 'Uppercase'] = uppercase
  syncus_df.loc[i*5 +2, 'Uppercase'] = uppercase
  syncus_df.loc[i*5 +3, 'Uppercase'] = uppercase
  syncus_df.loc[i*5 +4, 'Uppercase'] = uppercase

  all_caps = 1 if row["Tweet"].isupper() else 0
  custom_df.loc[i, 'All Caps'] = all_caps

  syncus_df.loc[i*5, 'All Caps'] = all_caps
  syncus_df.loc[i*5 +1, 'All Caps'] = all_caps
  syncus_df.loc[i*5 +2, 'All Caps'] = all_caps
  syncus_df.loc[i*5 +3, 'All Caps'] = all_caps
  syncus_df.loc[i*5 +4, 'All Caps'] = all_caps
        

In [ ]:
for i, row in syncus_df.iterrows():
  words = remove_stops_and_lemmatize(tokenize(row["Tweet"]))
  ex_words = remove_stops_and_lemmatize_keep_exclamation(tokenize(row["Tweet"]))

  for xword in ex_words:
    if xword == "!":
      syncus_df.loc[i, 'Exclamation'] = 1
      break

  for word in words:
    if word.lower() in abusive_lexicon.values:
      syncus_df.loc[i, 'Abusive'] = 1
      break

  length = len(row["Tweet"])
  syncus_df.loc[i, 'Length'] = length

  word_count = len(words)
  syncus_df.loc[i, 'Words'] = word_count
        

In [ ]:
def extract_by_word(df):
  for i, row in df.iterrows():
    words = tokenize(row["Tweet"])

    for word in words:
      if word.lower() in potensi_bias["0"].values:
        df.loc[i, word.lower()] = 1
      if word.lower() in abusive_lexicon["ABUSIVE"].values:
        df.loc[i, word.lower()] = 1
      if word.lower() in most_freq["0"].values:
        df.loc[i, word.lower()] = 1
  
  return df

In [ ]:
custom_df = extract_by_word(custom_df)
syncus_df = extract_by_word(syncus_df)

In [ ]:
custom_df

In [ ]:
syncus_df

In [ ]:
# custom_df.to_csv('all_extracted_custom_data.csv', encoding = 'latin-1') 
# files.download('all_extracted_custom_data.csv')

In [ ]:
# syncus_df.to_csv('all_extracted_synthetic_custom_data.csv', encoding = 'latin-1') 
# files.download('all_extracted_synthetic_custom_data.csv')

In [ ]:
for i, row in syncus_df.iterrows():
  new_row = row
  words = remove_stops_and_lemmatize(tokenize(row["Tweet"]))
  bias = []
  pool = ['r1', 'r2', 'r3', 'r4', 'r5']

  for word in words:
    if word.lower() in replacement_data["0"].values:
      bias.append(word)

  if len(bias) > 0:
    print(i, bias)

In [ ]:
replacement_data["0"].values